# 🐱🐶 Cat vs Dog Classifier - Gradio Server

This notebook loads the trained model and serves it through a Gradio web interface.

## Step 1: Import FastAI

In [ ]:
from fastai.vision.all import *
import os

## Step 2: Load the Trained Model

In [ ]:
# Load the trained model from the root directory
model_path = '../model.pkl'

if not os.path.exists(model_path):
    print(f"❌ Model file '{model_path}' not found!")
    print("Please run notebooks/dogs_cats.ipynb first to train and export the model.")
else:
    learn = load_learner(model_path)
    print("✅ Model loaded successfully!")
    print(f"📊 Model classes: {learn.dls.vocab}")

## Step 3: Define Prediction Function

In [ ]:
def predict(img):
    """Predict if image contains a cat or dog"""
    try:
        # Create PIL image
        img = PILImage.create(img)
        
        # Get prediction
        pred, pred_idx, probs = learn.predict(img)
        
        # Get labels
        labels = learn.dls.vocab
        
        # Create result dictionary
        result = {labels[i]: float(probs[i]) for i in range(len(labels))}
        
        return result
    except Exception as e:
        print(f"Prediction error: {e}")
        return {"Error": str(e)}

## Step 4: Create and Launch Gradio Interface

In [ ]:
import gradio as gr

# Create the interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🐱🐶 Cat vs Dog Classifier",
    description="Upload an image and I'll tell you if it's a cat or a dog!",
    examples=[
        ["../data/cat1.png"],
        ["../data/dog1.png"],
        ["../data/catdog.png"]
    ] if os.path.exists("../data") else None
)

# Launch the server
print("🚀 Starting Gradio server...")
interface.launch(
    share=True,
    server_name="0.0.0.0",
    show_error=True
)